In [27]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 200)

In [2]:
data = pd.read_csv('./Data_from2000.csv')
data.columns

Index(['FutCode', 'Date_', 'Open_', 'High', 'Low', 'Volume', 'Settlement',
       'OpenInterest', 'P', 'PB', 'PA', 'WA', 'CA', 'YD', 'HM', 'HA', 'MT',
       'LA', 'LM', 'OA', 'UP', 'ContrCode', 'ClsCode', 'DSMnem', 'LDB',
       'ContrDate', 'ContrDateFmt', 'ISOCurrCode', 'CurrUnitCode', 'UnitCode',
       'TrdStatCode', 'StartDate', 'LastTrdDate', 'SttlmntDate',
       'ExpirationDate', 'FirstNoticeDate', 'LastNoticeDate',
       'FirstDelvryDate', 'TickSizeUnitCode'],
      dtype='object')

In [3]:
##### Uncomment this to read from raw file and save as pickle #####
data = pd.read_csv(
    './Data_from2000.csv', 
    usecols=['FutCode','Date_','Open_', 'High', 'Low','Volume','CA','UP','DSMnem',
            'Settlement','OpenInterest','P','ContrCode','TrdStatCode',
            'StartDate', 'LastTrdDate', 'SttlmntDate','ExpirationDate'],
    dtype={'FutCode':int, 'Date_':str, 'Volume':float, 'P':float, 'CA':float, 'UP':float,'ContrCode':str,'DSMnem':str,'Open_':float, 'High':float, 'Low':float,
           'TrdStatCode':str,'StartDate':str,'LastTrdDate':str, 'SttlmntDate':str,'ExpirationDate':str})
data.to_pickle('./Data_from2000.pkl')

In [37]:
data = pd.read_pickle('./Data_from2000.pkl')

In [38]:
data

,FutCode,Date_,Open_,High,Low,Volume,Settlement,OpenInterest,P,CA,UP,ContrCode,DSMnem,TrdStatCode,StartDate,LastTrdDate,SttlmntDate,ExpirationDate
0,1,20140404,4.610000,4.610000,4.610000,0.0,4.610000,645.0,NaN,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
1,1,20150513,3.650999,3.650999,3.650999,0.0,3.650999,893.0,NaN,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
2,1,20170727,3.035999,3.035999,3.035999,48.0,3.044999,11081.0,2.0,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
3,1,20160516,3.074000,3.074000,3.074000,6.0,3.074000,1147.0,NaN,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
4,1,20180920,3.006000,3.006000,3.006000,3372.0,3.030999,34655.0,12.0,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27470602,443657,20221129,1907.500000,1907.500000,1907.500000,0.0,1907.500000,0.0,NaN,NaN,NaN,3223,NMG1024,A,20221128,20241029,23678,NaN
27470603,443658,20221129,3.618000,3.618000,3.618000,0.0,3.618000,0.0,NaN,NaN,NaN,2083,NQC1124,A,20221128,20241029,23678,NaN
27470604,443659,20221129,2.817000,2.817000,2.817000,0.0,2.817000,NaN,NaN,NaN,NaN,2085,NQH1223,A,20221129,20231129,23343,NaN
27470605,443660,20221129,2.180000,2.180000,2.180000,0.0,2.180000,NaN,NaN,NaN,NaN,2090,NQU1223,A,20221129,20231129,23343,NaN


In [40]:
data[data['ContrCode']=='2032']

,FutCode,Date_,Open_,High,Low,Volume,Settlement,OpenInterest,P,CA,UP,ContrCode,DSMnem,TrdStatCode,StartDate,LastTrdDate,SttlmntDate,ExpirationDate
11549,137,20000331,0.8980,0.9300,0.8815,18829.0,0.9169,NaN,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN
11550,137,20000329,0.9300,0.9380,0.8990,19510.0,0.9130,13106.0,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN
11551,137,20000328,0.9430,0.9660,0.9350,14221.0,0.9518,19553.0,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN
11552,137,20000327,0.9430,0.9620,0.9320,13040.0,0.9469,22448.0,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN
11553,137,20000323,0.9060,0.9210,0.9010,12950.0,0.9191,24209.0,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15186310,229247,20010301,0.7115,0.7115,0.7115,0.0,0.7115,66.0,NaN,NaN,NaN,2032,NHU0102,D,20010102,20011231,15341,NaN
15186311,229247,20010228,0.7080,0.7080,0.7080,0.0,0.7080,65.0,NaN,NaN,NaN,2032,NHU0102,D,20010102,20011231,15341,NaN
15186312,229247,20010305,0.7237,0.7237,0.7237,0.0,0.7237,66.0,NaN,NaN,NaN,2032,NHU0102,D,20010102,20011231,15341,NaN
15186313,229247,20010306,0.7296,0.7296,0.7296,2.0,0.7296,68.0,NaN,NaN,NaN,2032,NHU0102,D,20010102,20011231,15341,NaN


In [2]:
example_dictionary = {
        'Copper':'NHG0100',
        'Cotton':'NCT0319',
        'Cocoa':'NCC0919',
        'Corn':'CC.0503',
        'Wheat':'CW.0301',
        'Soybeans':'CS.0100',
        'Soy oil':'CBO0101',
        'Soy meat':'CMS0100',
        'Oats':'CO.0300',
        'Sugar':'NSB0103',
        'Pork bellies':'CPB0212',
        'Silver':'NSL0100',
        'Live cattle':'CLC0410',
        'Lean hogs':'CLH0200',
        'Orange juice':'NJO0100',
        'Platinum':'NPL0100',
        'Lumber':'ILB0109',
        'Feeder cattle':'CFC0100',
        'Coffee':'NKC0300',
        'Gold':'NGC0105',
        'Palladium':'NPA0112',
        'Heat oil':'NHO0103',
        'Crude oil':'NCL0609',
        'Gasoline unleaded':'NHU0400',
        'Gasoline blendstock':'NRB0610',
        'Rough rice':'CNR0102',
        'Natural gas':'NNG0119',
        'Milk':'CCM0102',
        'Coal':'NQL0614',
        'Butter':'CCB1219'
    }

In [3]:
def find_contracode_with_commodity_name(data, com_name, example_dictionary):
    """
    find contract code with given commodity_name.
    A contract code is corresponding to several contracts with different expiry date.
    """
    DSMnem = data['DSMnem'].values
    com_name_code = example_dictionary[com_name]
    if com_name_code in DSMnem:
        if len(data[data['DSMnem']==com_name_code]['ContrCode'].unique())!=1:
            print('--------------------')
            print(data[data['DSMnem']==com_name_code]['ContrCode'].unique())
            print('logic not correct')
            print('--------------------')
        return data[data['DSMnem']==com_name_code]['ContrCode'].iloc[0]
    else:
        print('None exist')


In [4]:
def extract_and_label_data(data, example_dictionary):
    # find contract code to commodity relations
    contract_code_to_commodity = {}
    print('find commodity name to contract code relation')
    for key in tqdm(example_dictionary):
        _code = find_contracode_with_commodity_name(data, key, example_dictionary)
        contract_code_to_commodity[_code] = key
    # extract selected commodity data and label it
    temp_data_lst = []
    print('extract and label data')
    for _code in tqdm(contract_code_to_commodity.keys()):
        _temp_data = data[data['ContrCode']==_code]
        _temp_data['Commodity_Type'] = contract_code_to_commodity[_code]
        temp_data_lst.append(_temp_data)
    data_selected = pd.concat(temp_data_lst, axis=0, ignore_index=True)
    return data_selected

In [9]:
data_selected = extract_and_label_data(data, example_dictionary)

find commodity name to contract code relation


100%|██████████| 30/30 [01:30<00:00,  3.03s/it]


extract and label data


100%|██████████| 30/30 [00:38<00:00,  1.28s/it]


In [5]:
#data_selected.to_pickle('./data_selected_from2000.pkl')
data_selected = pd.read_pickle('./data_selected_from2000.pkl')

In [6]:
def process_date(data):
    """Produce F1 F2 F3 F4 according to minimum time period requirement
    """
    df = data.copy()
    df['ttm'] = df.LastTrdDate.astype('datetime64[ns]') - df.Date_.astype('datetime64[ns]')
    df = df[df['ttm'] > pd.Timedelta('60 days')] # Select only contracts with more than 60 days to expiry
    df.sort_values(['Date_','ttm'], inplace=True)

    columns = ['f1','f2','f3','f4']
    results = df.reset_index(drop=True).groupby('Date_').apply(lambda s:s.iloc[:4])
    dfOutput = pd.DataFrame(index=results.index.get_level_values(0).drop_duplicates(), columns=columns)
    dfOutput = dfOutput.apply(lambda s: results.loc[s.name]['Settlement'].values, axis=1)
    dfOutput = pd.DataFrame(dfOutput.values.tolist(), index=dfOutput.index, columns=columns)

    return dfOutput

In [10]:
def final_result(data_selected):
    """
    Produce final result
    """
    all_contracts = data_selected['Commodity_Type'].drop_duplicates().values
    results = pd.DataFrame()

    for contract in all_contracts:
        dfSelected = data_selected[data_selected['Commodity_Type']==contract]
        dfTemp = process_date(dfSelected)
        dfTemp['contract'] = contract

        dfTemp.set_index('contract', append=True, inplace=True)
        dfTemp = dfTemp.reorder_levels(['contract', 'Date_'])

        if results.empty:
            results = dfTemp
        else:
            results = pd.concat([results, dfTemp])
    return results

In [36]:
data_selected[data_selected['Commodity_Type']=='Gasoline unleaded']

,FutCode,Date_,Open_,High,Low,Volume,Settlement,OpenInterest,P,CA,UP,ContrCode,DSMnem,TrdStatCode,StartDate,LastTrdDate,SttlmntDate,ExpirationDate,Commodity_Type
1956333,137,20000331,0.8980,0.9300,0.8815,18829.0,0.9169,NaN,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN,Gasoline unleaded
1956334,137,20000329,0.9300,0.9380,0.8990,19510.0,0.9130,13106.0,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN,Gasoline unleaded
1956335,137,20000328,0.9430,0.9660,0.9350,14221.0,0.9518,19553.0,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN,Gasoline unleaded
1956336,137,20000327,0.9430,0.9620,0.9320,13040.0,0.9469,22448.0,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN,Gasoline unleaded
1956337,137,20000323,0.9060,0.9210,0.9010,12950.0,0.9191,24209.0,NaN,NaN,NaN,2032,NHU0400,D,19990401,20000331,14703,NaN,Gasoline unleaded
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975194,229247,20010301,0.7115,0.7115,0.7115,0.0,0.7115,66.0,NaN,NaN,NaN,2032,NHU0102,D,20010102,20011231,15341,NaN,Gasoline unleaded
1975195,229247,20010228,0.7080,0.7080,0.7080,0.0,0.7080,65.0,NaN,NaN,NaN,2032,NHU0102,D,20010102,20011231,15341,NaN,Gasoline unleaded
1975196,229247,20010305,0.7237,0.7237,0.7237,0.0,0.7237,66.0,NaN,NaN,NaN,2032,NHU0102,D,20010102,20011231,15341,NaN,Gasoline unleaded
1975197,229247,20010306,0.7296,0.7296,0.7296,2.0,0.7296,68.0,NaN,NaN,NaN,2032,NHU0102,D,20010102,20011231,15341,NaN,Gasoline unleaded


In [11]:
# it takes about 2 minutes
contract_price = final_result(data_selected)

In [19]:
contract_price.index.get_level_values(0).unique()

Index(['Copper', 'Cotton', 'Cocoa', 'Corn', 'Wheat', 'Soybeans', 'Soy oil',
       'Soy meat', 'Oats', 'Sugar', 'Pork bellies', 'Silver', 'Live cattle',
       'Lean hogs', 'Orange juice', 'Platinum', 'Lumber', 'Feeder cattle',
       'Coffee', 'Gold', 'Palladium', 'Heat oil', 'Crude oil',
       'Gasoline unleaded', 'Gasoline blendstock', 'Rough rice', 'Natural gas',
       'Milk', 'Coal', 'Butter'],
      dtype='object', name='contract')

In [32]:
contract_price.reset_index().groupby('contract').count()

,Date_,f1,f2,f3,f4
contract,,,,,
Butter,4333,4331,4331,4331,4331
Coal,3739,3739,3700,3653,3608
Cocoa,5749,5749,5749,5749,5749
Coffee,5748,5748,5748,5748,5748
Copper,5761,5761,5759,5759,5759
Corn,3906,3905,3905,3905,3905
Cotton,5751,5750,5750,5750,5750
Crude oil,5758,5758,5757,5757,5756
Feeder cattle,3869,3865,3864,3864,3864


In [13]:
# index slicing
idx = pd.IndexSlice

copper_contract_price = contract_price.loc[idx['Copper',:]]

In [42]:
gasoline_contract_price = contract_price.loc[idx['Gasoline unleaded',:]]

In [44]:
gasoline_contract_price

,f1,f2,f3,f4
Date_,,,,
20000104,0.714000,0.7058,0.6929,0.6750
20000105,0.708500,0.7003,0.6875,0.6695
20000106,0.709000,0.7000,0.6865,0.6680
20000107,0.697000,0.6900,0.6790,0.6635
20000110,0.704800,0.6990,0.6895,0.6748
...,...,...,...,...
20061023,1.512400,NaN,NaN,NaN
20061024,1.559500,NaN,NaN,NaN
20061025,1.604799,NaN,NaN,NaN
